#Imports

In [1]:
!pip -q install transformers

     |████████████████████████████████| 5.5 MB 18.3 MB/s 
     |████████████████████████████████| 182 kB 56.5 MB/s 
     |████████████████████████████████| 7.6 MB 51.8 MB/s 


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
import torch
from tqdm import tqdm
import re
import  nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
tqdm.pandas()
import spacy
from scipy.special import softmax
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Data Cleaning

In [ ]:

# DEFAULT LISTS
SPECIAL_CHARCTERS_REGEX = re.compile("""[~` !@#$%\^&*\(\)\-_\+=\{\}\[\]\|\\/:;"'<>,\.\?]""")
SPECIAL_CHARCTERS = """~`!@#$%^&*()-_+={}[]|\/:;"'<>,.?"""
HASHTAG_REGEX = re.compile("#[a-zA-Z0-9]+")
EMAIL_REGEX = re.compile('\s[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}\s')

def celan_sentences(list_of_sentences, case=True, hashtag=True, special_char=True, special_chars=SPECIAL_CHARCTERS,
                     email=True,stopword=True,stopwords=[],stopwords_language = 'english',
                     root_form=True,root_method='Lemmatization'):
    
    """takes a list of sentences as input and return the same list of sentences after cleaning them
    current implementation with switches :
    1. lowercase
    2. removal of Hashtags
    3. removal of special characters 
    4. emails
    5. stopwords 
    6. reduction to root form using Lemmatization[default] or stemming 
    """
    stopwords = set(stopwords or nltk_stopwords.words(stopwords_language))
    cleaned_sentences = []
    for sentence in list_of_sentences:
        #TODO remove urls
        # sentence = sentence.lower()
        sentence = re.sub(HASHTAG_REGEX,'',sentence)
        sentence = re.sub(EMAIL_REGEX,'',sentence)
        sentence = "".join(filter(lambda x : x not in special_chars,sentence)).strip()
        sentence = " ".join([w for w in sentence.split(" ") if w not in stopwords ])
        cleaned_sentences.append(sentence)

    return cleaned_sentences

#ESG Model

In [3]:
# Grouping of classes 

dic = {'Business_Ethics' : "Labour_rights",
 'Data_Security' : "Customers",
 'Access_And_Affordability' : "Labour_rights",
 'Business_Model_Resilience' :'Customers',
 'Competitive_Behavior' : 'Customers',
 'Critical_Incident_Risk_Management' : "Governence",
 'Customer_Welfare' : "Customers",
 'Director_Removal' : "Governence",
 'Employee_Engagement_Inclusion_And_Diversity': "Labour_rights" ,
 'Employee_Health_And_Safety' : "Labour_rights",
 'Human_Rights_And_Community_Relations': 'Human_Rights_And_Community',
 'Labor_Practices' : 'Labour_rights',
 'Management_Of_Legal_And_Regulatory_Framework' : "Governence",
 'Physical_Impacts_Of_Climate_Change': "Environment",
 'Product_Quality_And_Safety' : "Customers",
 'Product_Design_And_Lifecycle_Management' : "Customers",
 'Selling_Practices_And_Product_Labeling' : "Customers",
 'Supply_Chain_Management' : "Labour_rights",
 'Systemic_Risk_Management' : "Governence",
 'Waste_And_Hazardous_Materials_Management' : "Environment",
 'Water_And_Wastewater_Management': "Environment",
 'Air_Quality' : "Environment",
 'Customer_Privacy' : "Customers",
 'Ecological_Impacts' : "Environment",
 'Energy_Management' : "Environment",
 'GHG_Emissions' : 'Environment'}

In [7]:
class predict_esg_class():
    def __init__(self):
        self.esg_pillars_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-esg", 
                                                                   max_length=512, truncation=True, padding="max_length")
        self.esg_pillars_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-esg")
        self.esg_subpillars_tokenizer = AutoTokenizer.from_pretrained('nbroad/ESG-BERT', 
                                                                      model_max_length =512, max_length=512, truncation=True, padding="max_length")
        self.esg_subpillars_model = AutoModelForSequenceClassification.from_pretrained('nbroad/ESG-BERT')
        self.esg_sentiment_tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment', 
                                                                     model_max_length =512, max_length=512, truncation=True, padding="max_length")
        self.esg_sentiment_model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
        self.esg_sentiment_fin_tokenizer = AutoTokenizer.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis', 
                                                                         model_max_length =512, max_length=512, truncation=True, padding="max_length")
        self.esg_sentiment_fin_model = AutoModelForSequenceClassification.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis')

        #self.summary_tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-3")
        #self.summary_model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-xsum-12-3")
        

    def esg_predict(self, story, model , tokenizer):
        story = story[:1000]
        inputs = tokenizer(story,return_tensors="pt")
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        return model.config.id2label[predicted_class_id]


    def predict(self, story, model_name):
        story = story[:1000]
        if model_name == "esg_pillars":
            return self.esg_predict(story, self.esg_pillars_model ,self.esg_pillars_tokenizer)
        elif model_name == "esg_subpillars":
            return self.esg_predict(story, self.esg_subpillars_model ,self.esg_subpillars_tokenizer)
        else:
            print(" Model not available")

    def sentiment(self, story, model_name):
      story = story[:700]
      if model_name == "esg_sentiment":
        inputs = self.esg_sentiment_tokenizer(story,return_tensors="pt")
        with torch.no_grad():
            logits = self.esg_sentiment_model(**inputs).logits
        return softmax(logits)
      elif model_name =='esg_fin_sentiment':
        inputs = self.esg_sentiment_fin_tokenizer(story,return_tensors="pt")
        with torch.no_grad():
            logits = self.esg_sentiment_fin_model(**inputs).logits
        return softmax(logits)
      else:
        return 'Check the Model'

    def summerize(self, text):
        inputs = self.summary_tokenizer(text, max_length=1024, return_tensors="pt", truncation=True)
        summary_ids = self.summary_model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=30)
        return self.summary_tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]




#Get Scale and Nature 

In [5]:
def get_scale_and_nature_of_impact(text):
    score = abc.sentiment(text,'esg_sentiment').tolist()
    score_fin = abc.sentiment(text,'esg_fin_sentiment').tolist()

    # print(score, score_fin)

    if score[0][0] >= .80:
        nature = 'Very Serious'
    elif score[0][0] > .70 and score[0][0] < .80:
        nature = 'Serious'
    elif score[0][0] > .60 and score[0][0] < .70:
        nature = 'Medium'
    else:
        nature = 'Minimal'

        
    if score_fin[0][0] >= .95:
        scale = 'Extremely Wide Spread'
    elif score_fin[0][0] > .90 and score_fin[0][0] < .95:
        scale = 'Extensive'
    elif score_fin[0][0] > .80 and score_fin[0][0] < .90:
        scale = 'Limited'
    else:
        scale = 'Low'
    
    # print(scale, nature)
    
    if scale == 'Extremely Wide Spread':
        if nature == 'Very Serious':
            return 'Very Severe'
        elif nature == 'Serious':
            return 'Severe'
        elif nature == 'Medium':
            return 'Severe'
        elif nature == 'Minimal':
            return 'Moderate'

    elif scale == 'Extensive':
        if nature == 'Very Serious':
            return 'Very Severe'
        elif nature == 'Serious':
            return 'Severe'
        elif nature == 'Medium':
            return 'Moderate'
        elif nature == 'Minimal':
            return 'Moderate'

    elif scale == 'Limited':
        if nature == 'Very Serious':
            return 'Severe'
        elif nature == 'Serious':
            return 'Moderate'
        elif nature == 'Medium':
            return 'Minor'
        elif nature == 'Minimal':
            return 'Minor'

    elif scale == 'Low':
        if nature == 'Very Serious':
            return 'Moderate'
        elif nature == 'Serious':
            return 'Moderate'
        elif nature == 'Medium':
            return 'Minor'
        elif nature == 'Minimal':
            return 'Minor'

In [8]:
abc = predict_esg_class()

#NER Model

In [9]:
ner_tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER",model_max_length =512, max_length=512, truncation=True, padding="max_length")

ner_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [10]:
def ner_predict(story):
    story = story[:1000]
    inputs = ner_tokenizer(story,return_tensors="pt")
    with torch.no_grad():
        logits = ner_model(**inputs).logits
    logits_argmax = logits.squeeze().argmax(dim=1)
    result = [ner_model.config.id2label[x] for x in logits_argmax.tolist()][1:-1]
    tokens = ner_tokenizer.tokenize(story)
    return ext_company_names(list(zip(result, tokens)))

    # return logits_argmax, logits            

In [11]:
def ext_company_names(ner_predict):
    companies = []
    temp = ''
    # print(ner_predict,len(ner_predict))
    for x , y in ner_predict:
        if x[-5:] == "B-ORG":
            if temp :
                companies.append(temp)
                temp = ''
            else :
                temp += y
        elif x[-5:] == "I-ORG":
            if temp:
                if y[:2] == '##':
                    temp += y[2:]
                elif y[0].isupper():
                    temp += ' ' +y
                else:
                    temp += y
        else:
            if temp:
                companies.append(temp)
                temp = ''
    if temp:
        companies.append(temp)
    return companies

#Scale of Impact

In [12]:
def find_impact(data):
    area = data.get('area')
    barrels = data.get('barrels')
    if area:
        if area > 100:
            return 'EWS'
        elif 9 < area < 100:
            return "EXT"
        else :
            return "LTD"
    elif barrels:
        if barrels > 60000:
            return 'EWS'
        elif 4999 < barrels < 60000:
            return "EXT"
        elif 1499 < barrels < 5000:
            return "LTD"
    else:
        return "LOW"


In [13]:
action_verbs = ['spilled','affected','spillage']

In [14]:
def parse_text(text_list,action_verbs):

    ret_dict = {
        'barrels': None,
        'area': None
    }

    ## token form spacy
    pos = spacy.load("en_core_web_sm")
    doc = [pos(text) for text in text_list]
    
    possible_data = []

    # find data form each 
    for item in doc:
        valid_verb = False
        temp = None
        for token in item:
            if token.dep_ in ['nummod',] :#and token.ent_type_ in ['QUANTITY']:
                temp = [token.head.text, token.text]
            elif token.text in action_verbs:
                valid_verb = True
            
        if valid_verb :
            possible_data.append(temp)
            temp = None
        else :
            temp = None


    return possible_data





In [15]:
def env_nature_of_impact(text_list,keywords=None):
    if keywords:
        pass
    else:
        keywords = {
            "dead": 'VS',
            "destruction": 'VS',
            "debilitation": 'S',
            "destruction": 'S',
            "damage" : 'S/M',
            "injury" : "S",
            "extinction" : 'VS'
        }
    ret_dict = {
        'barrels': None,
        'area': None
    }
    
    possible_data = []

    # find data form each 
    for item in text_list:

        for token in item.split(' '):
            if token in keywords.keys():
                  possible_data.append([keywords.get(token)])
            
    if possible_data:
        return possible_data
    else :
        return ["No keywords found"]

In [22]:
def check_ongoing_status(list_sent):
    ret_list = []
    nlp = spacy.load("en_core_web_sm")
    for item in list_sent:
        doc = nlp(item)
        for token in doc:
            if token.tag_ in ['VBD',] and token.dep_ == "ROOT":
                ret_list.append(f"Concluded")
            elif token.tag_ in ['VBG','VBZ'] and token.dep_ == "ROOT":
                ret_list.append(f"Ongoing")
            elif token.tag_ in ['VB','VBN','VBP'] and token.dep_ == "ROOT":
                ret_list.append(f"Partially Concluded")
    
    return Counter(ret_list).most_common(1)[0][0]

In [17]:
def controversy_score(controversy_assesment,company_role,status):
    if controversy_assesment == 'Very Severe':
        if company_role == 'Direct':
            if status == 'Ongoing':
                return 0
            elif status == 'Partially Concluded':
                return 1
            elif status == 'Concluded':
                return 2
        elif company_role == 'InDirect':
            if status == 'Ongoing':
                return 1
            elif status == 'Partially Concluded':
                return 2
            elif status == 'Concluded':
                return 3
    elif controversy_assesment == 'Severe':
        if company_role == 'Direct':
            if status == 'Ongoing':
                return 1
            elif status == 'Partially Concluded':
                return 2
            elif status == 'Concluded':
                return 3
        elif company_role == 'InDirect':
            if status == 'Ongoing':
                return 2
            elif status == 'Partially Concluded':
                return 3
            elif status == 'Concluded':
                return 4
    elif controversy_assesment == 'Moderate':
        if company_role == 'Direct':
            if status == 'Ongoing':
                return 4
            elif status == 'Partially Concluded':
                return 5
            elif status == 'Concluded':
                return 6
        elif company_role == 'InDirect':
            if status == 'Ongoing':
                return 5
            elif status == 'Partially Concluded':
                return 6
            elif status == 'Concluded':
                return 7
    elif controversy_assesment == 'Minor':
        if company_role == 'Direct':
            if status == 'Ongoing':
                return 6
            elif status == 'Partially Concluded':
                return 7
            elif status == 'Concluded':
                return 8
        elif company_role == 'InDirect':
            if status == 'Ongoing':
                return 7
            elif status == 'Partially Concluded':
                return 8
            elif status == 'Concluded':
                return 9

In [18]:
controversy_score('Moderate','Indirect','Concluded')

#Initialise Random Seed

In [19]:
def indier(x):
    return np.random.choice(["InDirect","Direct"] , p = [.1,.9])

#PipeLine

In [20]:
# load data 
data = pd.read_csv('/content/dataset1.csv')

data = data.iloc[1:100]

predictor = predict_esg_class()
# data = pd.read_csv('source.csv')

# data["title"] = data.Title.progress_apply(lambda x : x[0])
# data["story"] = data.title_story.apply(lambda x : ' '.join(celan_sentences(x[1])))
# data["real_story"] = data.News.progress_apply(lambda x : ' '.join(x[1]))

# data.drop('title_story',axis=1, inplace = True)

data["title_nwords"] = data.Title.progress_apply(lambda x : len(x.split(' ')))
data["story_nwords"] = data.News.progress_apply(lambda x : len(x.split(' ')))

data = data[(data.story_nwords > 75) & (data.title_nwords > 4)]

data = data.dropna()
data.drop(['title_nwords','story_nwords'],axis  = 1,inplace= True)



data['ESG_Pillar'] = data.News.progress_apply(lambda x : predictor.predict(x,'esg_pillars'))
data = data[data.ESG_Pillar != 'None']
data['ESG_Subpillar'] = data.News.progress_apply(lambda x : predictor.predict(x,'esg_subpillars'))
data['ESG_Subpillar2'] = data.ESG_Subpillar.progress_apply(lambda x : dic[x])

data['Organisations'] = data.News.progress_apply(lambda x : ner_predict(x))

data['Organisations'] = data.Organisations.progress_apply(lambda x : [x for x in list(set(x)) if len(x)> 2])
#data['NOE'] = data.News.progress_apply(lambda x : env_nature_of_impact(x))
data['Nature and Scale of Impact'] = data.News.progress_apply(get_scale_and_nature_of_impact)

100%|██████████| 86/86 [01:44<00:00,  1.22s/it]


#Is Concluded

In [23]:
data['is_concluded'] = data.News.progress_apply(lambda x : check_ongoing_status(x.split('. ')))

100%|██████████| 86/86 [01:31<00:00,  1.06s/it]


#Direct/Indirect

In [24]:
data['dir/indir'] = data.News.progress_apply(indier)

100%|██████████| 86/86 [00:00<00:00, 7957.08it/s]


#Controversy Assesment

In [25]:
data['CA'] = data.News.progress_apply(get_scale_and_nature_of_impact)

100%|██████████| 86/86 [01:50<00:00,  1.29s/it]


#Controversy Score

In [26]:
data['CS Score'] = data.apply(lambda x : controversy_score(x['CA'], x['dir/indir'],x['is_concluded'] ), axis =1 )

In [27]:
data

,Title,News,ESG_Pillar,ESG_Subpillar,ESG_Subpillar2,Organisations,Nature and Scale of Impact,is_concluded,dir/indir,CA,CS Score
1,Ecowas threatens tough sanctions on Guinea,Guinea-Ecowas-Transition-Sanctions Ecowas thre...,Social,Management_Of_Legal_And_Regulatory_Framework,Governence,"[UN General Assembly, Economic Communityof Wes...",Minor,Concluded,Direct,Minor,8
2,S/African court freezes pension payment for gr...,South Africa-Corruption-Lottery S/African cour...,Social,Business_Ethics,Labour_rights,"[APA, SIU, Lot, Special Tribunal, NLC, Nationa...",Minor,Concluded,Direct,Minor,8
3,Siply Plans To Launch 1000 Phygital Branches R...,The company targets to launch 200 branches in ...,Social,Employee_Engagement_Inclusion_And_Diversity,Labour_rights,[Siply],Minor,Ongoing,Direct,Minor,6
4,Denver judge rejects GOP residency complaint a...,A Democrat running to represent a state Senate...,Social,Management_Of_Legal_And_Regulatory_Framework,Governence,[Senate],Minor,Concluded,Direct,Minor,8
5,CT’s big hospital systems are buying up privat...,"Original reporting by Katy Golvala, Erica E. P...",Social,Competitive_Behavior,Customers,"[Yale New Haven, Hartford HealthCare—]",Minor,Partially Concluded,Direct,Minor,7
...,...,...,...,...,...,...,...,...,...,...,...
94,Simcoe County man sentenced in child luring case,A Simcoe County man will spend five years behi...,Social,Business_Ethics,Labour_rights,"[RCMP, Departmentof Homeland Security, Greyhou...",Minor,Concluded,InDirect,Minor,9
96,Govt issues notice on Benazir Ahmed's retireme...,US SANCTIONS\n\nThe police chief made headline...,Social,Business_Ethics,Labour_rights,"[US Treasury Department, RAB]",Minor,Concluded,InDirect,Minor,9
97,Regulatory Hurdles Slow Crypto Adoption in Ind...,Hostile regulatory environments see India and ...,Social,Systemic_Risk_Management,Governence,"[Chainalysis, Mala, AliPay]",Moderate,Partially Concluded,Direct,Moderate,5
98,AI startup Ada cuts 16 percent of its workforce,CEO cites tightening economy as Ada cuts 78 po...,Social,Systemic_Risk_Management,Governence,"[LinkedIn, Ada]",Moderate,Concluded,Direct,Moderate,6
